In [20]:
import pandas as pd
import re
from sklearn.utils import shuffle
import random as rand
import numpy as np
import math

In [21]:
random = 24

In [22]:
data = pd.read_csv('games.csv')
random = 24
data = data[["Metacritic", "Indie", "Presence", "Platform", 
             "Memory", "RatingsBreakdown", "ReleaseDate", "Soundtrack", 
             "OriginalCost", "DiscountedCost", "Controller",
             "Achievements", "Languages", "Storage", "Tags"]]

In [23]:
data.describe(include= 'all')

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,Languages,Storage,Tags
count,3356.000000,30045.000000,3.015600e+04,30123,28316,15044,27024,30045.000000,29504,727,29976.000000,30156.000000,30027,27491,30045
unique,NaN,NaN,NaN,2041,699,4806,4133,NaN,396,121,NaN,NaN,3196,2067,24866
top,NaN,NaN,NaN,PC,2 GB RAM,meh: 1,2018-08-22,NaN,$9.99,$0.49,NaN,NaN,English,1 GB available space,"Adventure, Casual, Hidden Object"
freq,NaN,NaN,NaN,15720,6487,910,238,NaN,4144,260,NaN,NaN,17133,2303,258
mean,72.924911,0.740722,8.822923e+04,NaN,NaN,NaN,NaN,0.004760,NaN,NaN,0.457666,41.047984,NaN,NaN,NaN
std,10.805296,0.438246,2.447597e+05,NaN,NaN,NaN,NaN,0.068826,NaN,NaN,0.498213,316.231316,NaN,NaN,NaN
min,20.000000,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,67.000000,0.000000,1.890000e+02,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
50%,74.000000,1.000000,3.380000e+02,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
75%,80.000000,1.000000,4.981500e+03,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,1.000000,21.000000,NaN,NaN,NaN


In [24]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

m RAM                      ' '1 GB RAM GB RAM'
 '2 GB MB RAM' '00 MB RAM' '1200 MB RAM' '270 MB ' '8 GB MB RAM'
 '512 MB of RAM or more ' '512 MB MB RAM' '120 MB RAM'
 '2GB                       '
 '512 MB of RAM for Windows(R) XP or 1 GB of RAM for Windows Vista(TM) '
 '512 Mb                     ' '500MB RAM ' '1024 MB '
 'WinXP - 1Gbyte RAM' '1 GB (2GB for Vista or Windows 7) '
 '256 MB RAM 512 MB RAM recommended) ' '512 MB ОЗУ MB RAM'
 '512MB (Vista 1GB)\t'
 '128Mb or more free system memory                     '
 '128 MB (256 MB on 2000/NT/XP) ' '512 МB RAM '
 '1 GB (2 GB recommended for Windows Vista and Windows 7) ' '17 MB RAM'
 '1GB System Memory ' 'Windows XP：1GB' '6094 MB RAM' '965 MB RAM'
 '1024 MB RAM ' '512 RAM ' '768 Mb MB RAM' '512m'
 '1 Gb RAM (2 Gb or greater recommended) ' '4 GB MB RAM' '64MB minimum '
 '2Gb VRAM MB RAM' '64 MB RAM (128 MB Recommended) '
 '2 GB of available system memory' '1GB RAM\t   \t ' '512 MB minimum'
 '128MB or higher' '512MB of System RA'
 '1 G

In [25]:
actions = {k:[] for k in data.columns}
actions['Indie'].append(lambda x: x == 1.0) 
actions['Controller'].append(lambda x: x == 1.0) 
actions['Soundtrack'].append(lambda x: x == 1.0)
actions['ReleaseDate'].append(lambda x: int(x.split('-')[0]))

def replace_MB(x):
    x = str(x).lower()
    first = re.search(r"\d", x)
    if (first == None):
        return None
    first = first.start()
    i = first+1
    while(i<len(x) and (x[i].isdigit() or x[i] == '.')):
        i = i+1
    last = i
    while(i<len(x) and x[i:i+2] != "gb" and x[i:i+2] != "mb"):
        i = i+1
    if (i == len(x) or x[i:i+2] == "gb"):
        return(float(x[first:last])*1000)
    return(float(x[first:last]))


def clean_originalCost(x):
    x = str(x).replace('$', '')
    try:
        number = float(x)
        return number
    except:
        return 0

def Metacritic_to_Range(x):
    if x > 75:
        return "Very good"
    
    elif x > 50 and x <= 75:
        return "Good"
    
    elif x > 25 and x <= 50:
        return "Bad"
    
    else:
        return "Very bad"
        
    

actions['Metacritic'].append(Metacritic_to_Range)
actions['OriginalCost'].append(lambda x: clean_originalCost(x))
actions['OriginalCost'].append(lambda x: float(str(x).replace('$', '')))
actions['DiscountedCost'].append(lambda x: float(str(x).replace('$', '')))
actions['Storage'].append(lambda x: replace_MB(x))
actions['Memory'].append(lambda x: replace_MB(x))

for name in actions:
    for action in actions[name]:
        data.loc[:,name] = data[data[name].notna()][name].apply(action)

data['DiscountedCost'] = data['DiscountedCost'].fillna(0)
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,Languages,Storage,Tags
0,Very good,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012.0,False,0.00,0.0,True,179.0,"English, Czech, Danish, Dutch, Finnish, French...",15000.0,"FPS, Shooter, Multiplayer, Competitive, Action..."
1,Very good,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017.0,False,0.00,0.0,True,61.0,"English, French, Italian, German, Japanese, Ko...",105000.0,"Free to Play, Looter Shooter, FPS, Multiplayer..."
2,Very good,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013.0,False,0.00,0.0,True,0.0,"English, Bulgarian, Czech, Danish, Dutch, Finn...",15000.0,"Free to Play, MOBA, Multiplayer, Strategy, eSp..."
3,Good,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014.0,False,19.99,0.0,True,0.0,"English, French, German, Russian",85000.0,"RPG, MMORPG, Open World, Massively Multiplayer..."
4,Good,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018.0,False,39.99,0.0,True,308.0,"English, French, Italian, German, Russian",50000.0,"Adventure, Action, Pirates, Open World, Multip..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018.0,False,0.99,0.0,True,0.0,English,10.0,"Indie, Casual"
30246,NaN,True,13.0,PC,6000.0,NaN,2017.0,False,0.99,0.0,False,0.0,English,2000.0,"Indie, Puzzle, Pixel Graphics, Mystery, Experi..."
30247,NaN,False,11.0,PC,2048.0,NaN,2018.0,False,1.99,0.0,False,0.0,English,2048.0,Casual
30248,NaN,True,423.0,PC,500.0,NaN,2017.0,False,0.99,0.0,True,0.0,English,300.0,"Action, Adventure, Indie, Casual"


In [26]:
platform = pd.Series(data['Platform']).str.get_dummies(', ')
platform.columns
data['Platform-Mobile'] = platform[['Android','iOS']].sum(axis = 1).apply(lambda x: x > 0) 
data['Platform-PC'] = platform[['Apple II','Atari ST', 'Classic Macintosh', 'Commodore / Amiga',  'Linux', 'PC', 'Web', 'macOS']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-Xbox'] = platform[['Xbox', 'Xbox 360', 'Xbox One']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-PlayStation'] = platform[['PS Vita', 'PSP', 'PlayStation', 'PlayStation 2', 'PlayStation 3', 'PlayStation 4']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-Nintendo'] = platform[['Game Boy', 'GameCube', 'Nintendo 3DS', 'Nintendo DS', 'Nintendo Switch', 'Wii', 'Wii U']].sum(axis = 1).apply(lambda x: x > 0) 
data['Platform-Sega'] = platform[['Dreamcast','Game Gear','Genesis','SEGA Master System', 'SEGA Saturn']].sum(axis = 1).apply(lambda x: x > 0) 
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,...,Achievements,Languages,Storage,Tags,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega
0,Very good,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012.0,False,0.00,0.0,...,179.0,"English, Czech, Danish, Dutch, Finnish, French...",15000.0,"FPS, Shooter, Multiplayer, Competitive, Action...",False,True,True,True,False,False
1,Very good,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017.0,False,0.00,0.0,...,61.0,"English, French, Italian, German, Japanese, Ko...",105000.0,"Free to Play, Looter Shooter, FPS, Multiplayer...",False,True,True,True,False,False
2,Very good,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013.0,False,0.00,0.0,...,0.0,"English, Bulgarian, Czech, Danish, Dutch, Finn...",15000.0,"Free to Play, MOBA, Multiplayer, Strategy, eSp...",False,True,False,False,False,False
3,Good,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014.0,False,19.99,0.0,...,0.0,"English, French, German, Russian",85000.0,"RPG, MMORPG, Open World, Massively Multiplayer...",False,True,False,False,False,False
4,Good,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018.0,False,39.99,0.0,...,308.0,"English, French, Italian, German, Russian",50000.0,"Adventure, Action, Pirates, Open World, Multip...",False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018.0,False,0.99,0.0,...,0.0,English,10.0,"Indie, Casual",False,True,False,False,False,False
30246,NaN,True,13.0,PC,6000.0,NaN,2017.0,False,0.99,0.0,...,0.0,English,2000.0,"Indie, Puzzle, Pixel Graphics, Mystery, Experi...",False,True,False,False,False,False
30247,NaN,False,11.0,PC,2048.0,NaN,2018.0,False,1.99,0.0,...,0.0,English,2048.0,Casual,False,True,False,False,False,False
30248,NaN,True,423.0,PC,500.0,NaN,2017.0,False,0.99,0.0,...,0.0,English,300.0,"Action, Adventure, Indie, Casual",False,True,False,False,False,False


In [27]:
def RatingsBreakdown(x):
    x = x.split(', ')
    result = [np.nan, np.nan, np.nan, np.nan]
    for i in x:
        if i.split(' ')[0] == 'recommended:':
            result[0] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'meh:':
            result[1] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'exceptional:':
            result[2] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'skip:':
            result[3] = float(i.split(' ')[1])
    return result  
        
df_aux = pd.DataFrame(data[data['RatingsBreakdown'].notna()]['RatingsBreakdown'].apply(lambda x: RatingsBreakdown(x)))
df_aux = pd.DataFrame(df_aux.RatingsBreakdown.values.tolist(),index = data[data['RatingsBreakdown'].notna()]['RatingsBreakdown'].index, columns=['RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip'])
data = data.join(df_aux)
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,...,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,RatingsBreakdown-Skip
0,Very good,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012.0,False,0.00,0.0,...,False,True,True,True,False,False,882.0,501.0,283.0,185.0
1,Very good,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017.0,False,0.00,0.0,...,False,True,True,True,False,False,726.0,465.0,227.0,106.0
2,Very good,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013.0,False,0.00,0.0,...,False,True,False,False,False,False,446.0,496.0,228.0,371.0
3,Good,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014.0,False,19.99,0.0,...,False,True,False,False,False,False,21.0,12.0,5.0,1.0
4,Good,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018.0,False,39.99,0.0,...,False,True,True,False,False,False,198.0,131.0,62.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018.0,False,0.99,0.0,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30246,NaN,True,13.0,PC,6000.0,NaN,2017.0,False,0.99,0.0,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30247,NaN,False,11.0,PC,2048.0,NaN,2018.0,False,1.99,0.0,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30248,NaN,True,423.0,PC,500.0,NaN,2017.0,False,0.99,0.0,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN


In [28]:
Tags = pd.Series(data['Tags']).str.get_dummies(', ').apply(lambda x: x > 0)
del Tags['Indie']
del Tags['Soundtrack']
del Tags['Controller']
Languages = pd.Series(data['Languages']).str.get_dummies(', ').apply(lambda x: x > 0)


#data = data.join(Tags)
#data = data.join(Languages)

In [29]:
del data['RatingsBreakdown']
del data['Platform']
del data['Tags']
del data['Languages']

In [30]:
for name in data.columns:
    print(name, data[name].isna().sum()/2000)

Metacritic 13.447
Indie 0.1025
Presence 0.047
Memory 0.9725
ReleaseDate 1.613
Soundtrack 0.1025
OriginalCost 0.373
DiscountedCost 0.0
Controller 0.137
Achievements 0.047
Storage 1.381
Platform-Mobile 0.0
Platform-PC 0.0
Platform-Xbox 0.0
Platform-PlayStation 0.0
Platform-Nintendo 0.0
Platform-Sega 0.0
RatingsBreakdown-Recommended 10.137
RatingsBreakdown-Meh 9.7875
RatingsBreakdown-Exceptional 11.1335
RatingsBreakdown-Skip 10.178


In [31]:
rand.seed(random)
data = data.dropna(subset=['Metacritic'], axis='index')
x = [y for y in range(len(data))]
x = rand.sample(x, 2200)
data = data.iloc[x]
data

,Metacritic,Indie,Presence,Memory,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,...,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,RatingsBreakdown-Skip
18317,Very good,False,887.0,256.0,2008.0,False,19.99,0.0,False,0.0,...,True,True,False,False,False,False,12.0,9.0,3.0,7.0
4767,Good,True,204071.0,2000.0,2014.0,False,19.99,0.0,False,79.0,...,False,True,False,False,False,False,2.0,1.0,NaN,2.0
10520,Good,False,13512.0,2000.0,2016.0,False,14.99,0.0,False,50.0,...,False,True,False,False,False,False,4.0,1.0,NaN,1.0
1579,Good,False,997391.0,6000.0,2017.0,False,39.99,0.0,True,141.0,...,False,True,True,True,False,False,39.0,33.0,6.0,19.0
2055,Very good,False,75371.0,2000.0,2016.0,False,14.99,0.0,True,64.0,...,True,True,False,True,False,False,113.0,49.0,68.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,Very good,False,265356.0,2000.0,2014.0,False,39.99,0.0,False,90.0,...,False,True,False,False,False,False,97.0,61.0,17.0,12.0
16926,Good,False,54700.0,2000.0,2012.0,False,9.99,0.0,True,21.0,...,False,True,False,False,False,False,3.0,20.0,1.0,7.0
3098,Good,False,1000528.0,1000.0,2012.0,False,9.99,0.0,True,24.0,...,True,True,True,True,False,False,25.0,22.0,NaN,8.0
4913,Very good,True,12592.0,1000.0,2012.0,False,9.99,0.0,True,27.0,...,False,True,False,False,False,False,25.0,17.0,4.0,6.0


In [32]:
for col in data.columns:
    if len(data[col].unique()) == 1:
        data.drop(col,inplace=True,axis=1)

In [33]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

Metacritic
['Very good' 'Good' 'Bad' 'Very bad']
Indie
[False True]
Presence
[8.870000e+02 2.040710e+05 1.351200e+04 ... 1.000528e+06 1.259200e+04
 5.049180e+05]
Memory
[2.560e+02 2.000e+03 6.000e+03       nan 4.000e+03 5.120e+02 1.000e+03
 8.000e+03 2.048e+03 2.500e+03 1.500e+03 1.024e+03 3.000e+03 6.400e+01
 7.680e+02 1.280e+02 2.000e+02 1.536e+03 3.000e+02 1.600e+01 8.192e+03
 4.000e+00 4.096e+03 9.600e+01 8.000e+02 3.072e+03 5.000e+02 2.000e+00
 2.000e+06 4.000e+02 2.560e+05 3.000e+00 3.200e+01 8.000e+00 5.000e+03
 5.120e+05 2.500e+02 1.600e+04 4.000e+01 1.000e+02 1.200e+03 6.144e+03
 3.840e+02 1.200e+04 5.200e+02]
ReleaseDate
[2008. 2014. 2016. 2017. 2013. 2018. 2015. 2007. 2009. 2020. 2019. 2012.
 2010. 2006. 2004. 1999. 2002. 2011. 2005. 2001. 1994. 1993. 1998. 2003.
 1996. 2000.   nan 1997.]
Soundtrack
[False True]
OriginalCost
[19.99 14.99 39.99  9.99  0.   29.99 13.99  4.99  5.99 12.99 59.99  0.99
  7.99  6.99 24.99 49.99  3.99 11.99   nan  2.99  8.99 34.99  1.99 10.99
 15.99

In [34]:
from scipy import stats

In [35]:
col =['Presence', 'Memory', 'ReleaseDate', 'Achievements', 'Storage']
out = None
for name in col:
    data = data[~(data[name] - data[name].mean() > 3*data[name].std())]

In [36]:
data

,Metacritic,Indie,Presence,Memory,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,...,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,RatingsBreakdown-Skip
18317,Very good,False,887.0,256.0,2008.0,False,19.99,0.0,False,0.0,...,True,True,False,False,False,False,12.0,9.0,3.0,7.0
4767,Good,True,204071.0,2000.0,2014.0,False,19.99,0.0,False,79.0,...,False,True,False,False,False,False,2.0,1.0,NaN,2.0
10520,Good,False,13512.0,2000.0,2016.0,False,14.99,0.0,False,50.0,...,False,True,False,False,False,False,4.0,1.0,NaN,1.0
1579,Good,False,997391.0,6000.0,2017.0,False,39.99,0.0,True,141.0,...,False,True,True,True,False,False,39.0,33.0,6.0,19.0
2055,Very good,False,75371.0,2000.0,2016.0,False,14.99,0.0,True,64.0,...,True,True,False,True,False,False,113.0,49.0,68.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,Very good,False,265356.0,2000.0,2014.0,False,39.99,0.0,False,90.0,...,False,True,False,False,False,False,97.0,61.0,17.0,12.0
16926,Good,False,54700.0,2000.0,2012.0,False,9.99,0.0,True,21.0,...,False,True,False,False,False,False,3.0,20.0,1.0,7.0
3098,Good,False,1000528.0,1000.0,2012.0,False,9.99,0.0,True,24.0,...,True,True,True,True,False,False,25.0,22.0,NaN,8.0
4913,Very good,True,12592.0,1000.0,2012.0,False,9.99,0.0,True,27.0,...,False,True,False,False,False,False,25.0,17.0,4.0,6.0


In [37]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

Metacritic
['Very good' 'Good' 'Bad' 'Very bad']
Indie
[False True]
Presence
[8.870000e+02 2.040710e+05 1.351200e+04 ... 1.000528e+06 1.259200e+04
 5.049180e+05]
Memory
[2.560e+02 2.000e+03 6.000e+03       nan 4.000e+03 5.120e+02 1.000e+03
 8.000e+03 2.048e+03 2.500e+03 1.500e+03 1.024e+03 3.000e+03 6.400e+01
 7.680e+02 1.280e+02 2.000e+02 1.536e+03 3.000e+02 1.600e+01 8.192e+03
 4.000e+00 4.096e+03 9.600e+01 8.000e+02 3.072e+03 5.000e+02 2.000e+00
 4.000e+02 3.000e+00 3.200e+01 8.000e+00 5.000e+03 2.500e+02 1.600e+04
 4.000e+01 1.000e+02 1.200e+03 6.144e+03 3.840e+02 1.200e+04 5.200e+02]
ReleaseDate
[2008. 2014. 2016. 2017. 2013. 2018. 2015. 2007. 2009. 2020. 2019. 2012.
 2010. 2006. 2004. 1999. 2002. 2011. 2005. 2001. 1994. 1993. 1998. 2003.
 1996. 2000.   nan 1997.]
Soundtrack
[False True]
OriginalCost
[19.99 14.99 39.99  9.99  0.   29.99 13.99  4.99  5.99 12.99 59.99  0.99
  7.99  6.99 24.99 49.99  3.99 11.99   nan  2.99  8.99 34.99  1.99 10.99
 15.99 13.37 17.99 44.99 16.99  1.29 

In [38]:
data.to_csv('games_clean.csv', index=False)